Install `xlrd` for reading the `xls` file

In [58]:
# %conda install xlrd==2.0.1
# $ conda install -c conda-forge py-xgboost-gpu


Set the path to the `xls` file

In [1]:
training_file = "../TrainDataset2024.xls"
# training_file = "/kaggle/input/dataset/TrainDataset2024.xls"

Import libraries

In [2]:
import sys
import os

# Add the parent directory to the system path
sys.path.append(os.path.abspath('../'))  # Adjust the path as needed

from my_util import df_to_corr_matrix, remove_outliers

import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from matplotlib.colors import Normalize
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, cross_val_predict, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, make_scorer, balanced_accuracy_score
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif, chi2, mutual_info_classif
from sklearn.impute import KNNImputer


from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

from joblib import Parallel, delayed

import xgboost as xgb
from xgboost import XGBClassifier

from pickle import dump , load

import warnings

2024-12-05 00:35:47.391919: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 00:35:47.557489: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733358947.620102    1213 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733358947.638547    1213 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 00:35:47.796076: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Read the data into X and y

In [3]:
NUM_OF_SELECTED_FEATURES = "corr_25"

data = pd.read_excel(training_file)
data.replace(999, np.nan, inplace=True)

data.drop(["ID", "RelapseFreeSurvival (outcome)"], axis=1, inplace=True)
data.dropna(subset=["pCR (outcome)"], inplace=True)

with open(f'../FeatureSelection/pkl/{NUM_OF_SELECTED_FEATURES}_selected_features.pkl', mode='rb') as file:
    selected_features = load(file)
    print(f"Loaded '{file.name}' to selected_feature")

X = data[selected_features]
y = data["pCR (outcome)"]
print(X.shape, y.shape)

print(selected_features)

Loaded '../FeatureSelection/pkl/corr_25_selected_features.pkl' to selected_feature
(395, 25) (395,)
['Gene', 'HER2', 'PgR', 'ER', 'original_firstorder_10Percentile', 'original_ngtdm_Busyness', 'LNStatus', 'TumourStage', 'original_gldm_DependenceEntropy', 'original_firstorder_Skewness', 'original_glrlm_ShortRunHighGrayLevelEmphasis', 'original_ngtdm_Strength', 'original_gldm_SmallDependenceEmphasis', 'original_firstorder_InterquartileRange', 'original_shape_MajorAxisLength', 'original_glrlm_LongRunLowGrayLevelEmphasis', 'original_firstorder_Minimum', 'HistologyType', 'ChemoGrade', 'original_shape_Maximum2DDiameterRow', 'original_shape_Maximum2DDiameterColumn', 'original_shape_SurfaceVolumeRatio', 'original_shape_LeastAxisLength', 'original_glcm_Autocorrelation', 'original_shape_Sphericity']


In [62]:
# # Set up the matplotlib figure
# plt.figure(figsize=(40, 30))

# # Loop through each feature to create a scatter plot
# for i, feature in enumerate(X.columns):
#     plt.subplot(5, 6, i + 1)  # Adjust the number of rows and columns based on the number of features
#     sns.scatterplot(x=y, y=X[feature], hue=y, style=y, palette='Set2', alpha=0.7)
#     plt.title(feature)
#     plt.xlabel('pCR (outcome)')
#     plt.ylabel(feature)
#     plt.xlim(-2, 3)

# plt.tight_layout()
# plt.show()

In [63]:
# df_to_corr_matrix(X, size_factor=1.6, sep=150)

### Split the data into train_full and test_reserved (untouch)

In [4]:
# Close ratio random_state
# [14, 47, 49, 52, 62, 76, 83, 89, 92, 116, 118, 122, 136, 138, 144, 146, 150, 156, 157, 159, 170, 172, 174, 185]

while True:  
    X_train_full, X_test_reserved, y_train_full, y_test_reserved = train_test_split(X, y, test_size=0.2, random_state=14) # similar distribution of 1 and 0
    # X_train_full, X_test_reserved, y_train_full, y_test_reserved = train_test_split(X, y, test_size=0.2, random_state=None)

    X_train_full.reset_index(drop=True, inplace=True)
    X_test_reserved.reset_index(drop=True, inplace=True)
    y_train_full.reset_index(drop=True, inplace=True)
    y_test_reserved.reset_index(drop=True, inplace=True)

    ratio_train = sum(y_train_full[y_train_full==1]) / len(y_train_full)
    ratio_test = sum(y_test_reserved[y_test_reserved==1]) / len(y_test_reserved)

    if abs(ratio_train - ratio_test) < 0.1:
        break

print("Splited the data into train and test. The test will not be used in the training, but just for test the xgb. ")
print(f"The training data has {len(X_train_full)} data. The testing data has {len(X_test_reserved)} data. ")
print(f"Positive ratio: \n\tTrain: {ratio_train:.5f}\n\tTest: {ratio_test:.5f}")

Splited the data into train and test. The test will not be used in the training, but just for test the xgb. 
The training data has 316 data. The testing data has 79 data. 
Positive ratio: 
	Train: 0.21203
	Test: 0.21519


### Outliers

In [65]:
# # The result of keeping outliers is better
# X_train_full, y_train_full = remove_outliers(X_train_full, y_train_full, selected_features)

### XGBoost

In [5]:
print(X_train_full.shape)
print(y_train_full.shape)

(316, 25)
(316,)


In [8]:
model = XGBClassifier(objective="binary:logistic")

# param_grid = {
#     "gamma": [0.2, 0.3],
#     "learning_rate": [0.3, 0.5],
#     "max_bin": [2, 5, 10, 20],
#     "max_depth": [1, 2, 3],
#     "max_leaves": [1, 2, 3, 4],
#     "n_estimators": [5, 10, 20, 30, 40, 50],
#     "scale_pos_weight": [4.5],  # imbalanced data
# }
param_grid = {
    "gamma": [0, 0.01, 0.1, 0.3],
    "learning_rate": [0.2, 0.25, 0.3, 0.4],
    "max_bin": [2, 4, 5, 6, 8, 10],
    "max_depth": [1, 2, 3],
    "max_leaves": [1, 2, 3],
    "min_child_weight": [0, 0.001, 0.005, 0.01, 1],
    "n_estimators": [30, 50, 70, 75, 80, 100],
    "num_parallel_tree": [1],
    "scale_pos_weight": [3.8, 4.5],  # imbalanced data
}

best_score = 0
best_score_at = None

for i in range(18):

    kf = StratifiedKFold(5, shuffle=True, random_state=i)

    # Set up the GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring={
            "f1": "f1",
            "recall": "recall",
            "specificity": make_scorer(recall_score, pos_label=0),
            "precision": "precision",
            "balanced_accuracy_score": make_scorer(balanced_accuracy_score),
        },
        cv=kf,
        # cv=5,
        verbose=1,
        n_jobs=-1,
        return_train_score=True,
        refit="balanced_accuracy_score",
    )

    # Fit the model
    grid_search.fit(X_train_full, y_train_full)

    # Get the best parameters and best score
    result = pd.DataFrame(grid_search.cv_results_)
    best_params = grid_search.best_params_
    best_index = grid_search.best_index_
    best_f1 = result["mean_test_f1"][best_index]
    best_precision = result["mean_test_precision"][best_index]
    best_recall = result["mean_test_recall"][best_index]
    best_specificity = result["mean_test_specificity"][best_index]
    best_balanced_accuracy_score = result["mean_test_balanced_accuracy_score"][best_index]

    print(f"random_state: {i}")
    print(f"Best Parameters at Index {best_index} :", best_params)
    print(f"Balanced accuracy score: {best_balanced_accuracy_score}")
    print(f"F1 Score: {best_f1}")
    print(f"Precision Score: {best_precision}")
    print(f"Recall Score: {best_recall}")
    print(f"Specificity Score: {best_specificity}")
    print()

    if best_balanced_accuracy_score > best_score:
        best_score = best_balanced_accuracy_score
        best_score_at = i

    pd.DataFrame(grid_search.cv_results_).to_csv(f"output{i}.csv")

print(f"Best run is the {best_score_at}th run. Score: {best_score}")

Fitting 5 folds for each of 51840 candidates, totalling 259200 fits
random_state: 0
Best Parameters at Index 8161 : {'gamma': 0, 'learning_rate': 0.3, 'max_bin': 6, 'max_depth': 1, 'max_leaves': 2, 'min_child_weight': 0, 'n_estimators': 30, 'num_parallel_tree': 1, 'scale_pos_weight': 4.5}
Balanced accuracy score: 0.7762323390894819
F1 Score: 0.5732413323110996
Precision Score: 0.42922841143530793
Recall Score: 0.8659340659340661
Specificity Score: 0.686530612244898

Fitting 5 folds for each of 51840 candidates, totalling 259200 fits
random_state: 1
Best Parameters at Index 3843 : {'gamma': 0, 'learning_rate': 0.25, 'max_bin': 4, 'max_depth': 1, 'max_leaves': 2, 'min_child_weight': 0, 'n_estimators': 50, 'num_parallel_tree': 1, 'scale_pos_weight': 4.5}
Balanced accuracy score: 0.763171114599686
F1 Score: 0.5623509823509825
Precision Score: 0.4324091925871681
Recall Score: 0.823076923076923
Specificity Score: 0.703265306122449

Fitting 5 folds for each of 51840 candidates, totalling 2592

In [68]:
model = grid_search.best_estimator_
model.save_model("model.ubj")

In [69]:
results = pd.DataFrame(grid_search.cv_results_)
result_start = 4
print(list(results.keys())[result_start:result_start+len(param_grid)])


['param_gamma', 'param_learning_rate', 'param_max_bin', 'param_max_depth', 'param_max_leaves', 'param_min_child_weight', 'param_n_estimators', 'param_num_parallel_tree', 'param_scale_pos_weight']


In [ ]:
results = pd.DataFrame(grid_search.cv_results_)

# filter = results['param_num_parallel_tree'] == 10
filter = pd.Series([True] * len(results)) # include all data

fig = go.Figure()
# Add mean train score trace
fig.add_trace(go.Scatter(
    x=list(range(len(results["mean_train_f1"][filter]))),
    y=results["mean_train_f1"][filter],
    mode='lines+markers',
    name='Mean Train F1',
    text=list(results['params'][filter]),  # Display parameter values on hover
    hoverinfo='text+y+x',  # Show parameter values and y value
))
fig.add_trace(go.Scatter(
    x=list(range(len(results["mean_train_recall"][filter]))),
    y=results["mean_train_recall"][filter],
    mode='lines+markers',
    name='Mean Train Recall',
    text=list(results['params'][filter]),  # Display parameter values on hover
    hoverinfo='text+y+x',  # Show parameter values and y value
    visible="legendonly",
))
fig.add_trace(go.Scatter(
    x=list(range(len(results["mean_train_specificity"][filter]))),
    y=results["mean_train_specificity"][filter],
    mode='lines+markers',
    name='Mean Train Specificity',
    text=list(results['params'][filter]),  # Display parameter values on hover
    hoverinfo='text+y+x',  # Show parameter values and y value
    visible="legendonly",
))
# Add mean test score trace
fig.add_trace(go.Scatter(
    x=list(range(len(results["mean_test_f1"][filter]))),
    y=results["mean_test_f1"][filter],
    mode='lines+markers',
    name='Mean Test F1',
    text=list(results['params'][filter]),  # Display parameter values on hover
    hoverinfo='text+y+x',  # Show parameter values and y value
))
fig.add_trace(go.Scatter(
    x=list(range(len(results["mean_test_recall"][filter]))),
    y=results["mean_test_recall"][filter],
    mode='lines+markers',
    name='Mean Test Recall',
    text=list(results['params'][filter]),  # Display parameter values on hover
    hoverinfo='text+y+x',  # Show parameter values and y value
    visible="legendonly",
))
fig.add_trace(go.Scatter(
    x=list(range(len(results["mean_test_specificity"][filter]))),
    y=results["mean_test_specificity"][filter],
    mode='lines+markers',
    name='Mean Test Specificity',
    text=list(results['params'][filter]),  # Display parameter values on hover
    hoverinfo='text+y+x',  # Show parameter values and y value
    visible="legendonly",
))

# Update layout
fig.update_layout(
    title='Grid Search Mean Train and Test Scores',
    xaxis_title='Parameter Combinations (Index)',
    yaxis_title='Score',
    legend_title='Scores',
    hovermode='closest'
)
fig.show()

In [71]:
model = grid_search.best_estimator_

X_test = X_test_reserved

y_pred = model.predict(X_test)
report = classification_report(y_test_reserved, y_pred)
cm = confusion_matrix(y_test_reserved, y_pred)

print("Testing set:")
print(X_test_reserved.shape)
print(report)
print(cm)
print()
print(f"Balanced accuracy score: {balanced_accuracy_score(y_test_reserved, y_pred)}")
print(f"F1 Score: {f1_score(y_test_reserved, y_pred)}")
print(f"Precision: {precision_score(y_test_reserved, y_pred)}")
print(f"Recall: {recall_score(y_test_reserved, y_pred)}")
print(f"Specificity: {recall_score(y_test_reserved, y_pred, pos_label=0)}")

while True:  
    X_train_full, X_test_reserved, y_train_full, y_test_reserved = train_test_split(X, y, test_size=0.2, random_state=None)

    X_train_full.reset_index(drop=True, inplace=True)
    X_test_reserved.reset_index(drop=True, inplace=True)
    y_train_full.reset_index(drop=True, inplace=True)
    y_test_reserved.reset_index(drop=True, inplace=True)

    ratio_train = sum(y_train_full[y_train_full==1]) / len(y_train_full)
    ratio_test = sum(y_test_reserved[y_test_reserved==1]) / len(y_test_reserved)

    if abs(ratio_train - ratio_test) < 0.01:
        break

print()
print("Splited the data into train and test. The test will not be used in the training, but just for test the xgb. ")
print(f"The training data has {len(X_train_full)} data. The testing data has {len(X_test_reserved)} data. ")
print(f"Positive ratio: \n\tTrain: {ratio_train:.5f}\n\tTest: {ratio_test:.5f}")

# print(f"TARGET_NUM_OF_FEATURES: {TARGET_NUM_OF_FEATURES}, scaler: {SCALER}, num_of_features: {num_of_features}")

print(f"Best Parameters at Index {best_index} :", best_params)
print(f"Balanced accuracy score: {best_balanced_accuracy_score}")
print(f"F1 Score: {best_f1}")
print(f"Precision Score: {best_precision}")
print(f"Recall Score: {best_recall}")
print(f"Specificity Score: {best_specificity}")

model.fit(X_train_full, y_train_full)



y_pred = model.predict(X_train_full)
report = classification_report(y_train_full, y_pred)
cm = confusion_matrix(y_train_full, y_pred)

print("\nTraining set:")
print(X_train_full.shape)
print(report)
print(cm)
print()

X_test = X_test_reserved

y_pred = model.predict(X_test)
report = classification_report(y_test_reserved, y_pred)
cm = confusion_matrix(y_test_reserved, y_pred)

print("Testing set:")
print(X_test_reserved.shape)
print(report)
print(cm)
print()
print(f"Balanced accuracy score: {balanced_accuracy_score(y_test_reserved, y_pred)}")
print(f"F1 Score: {f1_score(y_test_reserved, y_pred)}")
print(f"Precision: {precision_score(y_test_reserved, y_pred)}")
print(f"Recall: {recall_score(y_test_reserved, y_pred)}")
print(f"Specificity: {recall_score(y_test_reserved, y_pred, pos_label=0)}")



Testing set:
(79, 25)
              precision    recall  f1-score   support

         0.0       0.95      0.60      0.73        62
         1.0       0.38      0.88      0.53        17

    accuracy                           0.66        79
   macro avg       0.66      0.74      0.63        79
weighted avg       0.83      0.66      0.69        79

[[37 25]
 [ 2 15]]

Balanced accuracy score: 0.7395635673624288
F1 Score: 0.5263157894736842
Precision: 0.375
Recall: 0.8823529411764706
Specificity: 0.5967741935483871

Splited the data into train and test. The test will not be used in the training, but just for test the xgb. 
The training data has 316 data. The testing data has 79 data. 
Positive ratio: 
	Train: 0.21203
	Test: 0.21519
Best Parameters at Index 1112 : {'gamma': 0, 'learning_rate': 0.2, 'max_bin': 8, 'max_depth': 1, 'max_leaves': 2, 'min_child_weight': 0, 'n_estimators': 70, 'num_parallel_tree': 1, 'scale_pos_weight': 4.5}
Balanced accuracy score: 0.7834536891679749
F1 Score: 0

In [72]:
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True)

y_pred_cv = cross_val_predict(model, X, y, cv=stratified_kfold)

# Calculate evaluation metrics
f1 = f1_score(y, y_pred_cv)
precision = precision_score(y, y_pred_cv)
recall = recall_score(y, y_pred_cv)
specificity = recall_score(y, y_pred_cv, pos_label=0)

# Print evaluation metrics
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity)

print(confusion_matrix(y, y_pred_cv))

F1 Score: 0.5403225806451613
Precision: 0.40853658536585363
Recall: 0.7976190476190477
Specificity: 0.6881028938906752
[[214  97]
 [ 17  67]]


`Best Parameters at Index 244 : {'gamma': 0, 'learning_rate': 0.3, 'max_bin': 5, 'max_depth': 1, 'max_leaves': 2, 'min_child_weight': 0.001, 'n_estimators': 75, 'num_parallel_tree': 1, 'scale_pos_weight': 4.5}`

`Best Parameters at Index 1112 : {'gamma': 0, 'learning_rate': 0.2, 'max_bin': 8, 'max_depth': 1, 'max_leaves': 2, 'min_child_weight': 0, 'n_estimators': 70, 'num_parallel_tree': 1, 'scale_pos_weight': 4.5}`

`random_state=13` `{'gamma': 0, 'learning_rate': 0.3, 'max_bin': 6, 'max_depth': 1, 'max_leaves': 2, 'min_child_weight': 0, 'n_estimators': 30, 'num_parallel_tree': 1, 'scale_pos_weight': 4.5}`
